# Introduction

In this notebook we will use the Lorentz force equation,
$$ \vec{F} = q\left(\vec{E}+\vec{v}\times\vec{B}\right) \tag{Lorentz force}$$
and the Euler's formula
$$ e^{i\theta} = \cos\theta + i\sin\theta. \tag{Euler formula}$$
The subject of this notebook is covered in the bibliography in the following chapters:

* Chen<cite data-cite="chen1974"></cite>: chapter Two, section 2.2
* Nicholson<cite data-cite="nicholson1983"></cite>: chapter 2, section 2.2
* Bittencourt<cite data-cite="Bittencourt2004"></cite>: chapter 2
* Goldston<cite data-cite="Goldston1995"></cite>: chapter 2

The examples are prepared with the help of two scientific software packages, *Scipy*<cite data-cite="jones2001"></cite> and *IPython*<cite data-cite="PER-GRA:2007"></cite>.

# Movement without $\vec{E}$ field

Let's keep it simple: $\vec{B}(\vec{r},t) = B_0\vec{u}_\parallel;\quad \vec{v}_0 = v_{\perp,0}\vec{u}_\perp+v_{z,0}\vec{u}_\parallel$ and we take $\vec{u}_\parallel \equiv \vec{u}_z$.

* The Lorentz force is 
$$ \vec{F} = q\vec{v}\times\vec{B}$$
and, as $\vec{F} \perp \vec{v} \Rightarrow$ constant kinetic energy, $W$.

## How is the movement?

$\color{red}{Discussion...}$

From the Lorentz force equation in cartesian coordinates, we write
\begin{align}
\dot{v}_x &= \frac{q}{m}v_y B_0 \\
\dot{v}_y &= -\frac{q}{m}v_x B_0 \\
\dot{v}_z &= 0
\end{align}
Taking the derivative of any of the first two equations we obtain the homogeneous equation for a *harmonic oscilator*:
$$ \ddot{v}_{x,y} + \left(\frac{q}{m}B_0\right)^2 v_{x,y} = 0 $$

$\color{red}{Discussion...}$

Integrating we obtain
\begin{align}
v_x &= v_\perp \exp(i{\color{green}{\omega_c}}t+i\delta) \\
v_y &= \pm i v_\perp \exp(i{\color{green}{\omega_c}}t+i\delta) \\
v_z &= v_{z,0}
\end{align}
with $v_\perp=(v_{x,0}^2+v_{y,0}^2)^{1/2}$, $\delta=\mp\tan^{-1}(v_{y,0}/v_{x,0})$, the signal in $\pm$ depends on the charge $q$ and $\boxed{\color{green}{\omega_c \equiv \frac{|q|B}{m}}}$ is the **cyclotron frequency**.

   * Uniform circular motion in $\perp$;
   * $v_\parallel$ is constant $\Rightarrow$ uniform motion in $\parallel$.

Integrating again we obtain
\begin{align}
x &= x_0 - i{\color{red}{(v_\perp/\omega_c)}} \left[\exp(i{\color{green}{\omega_c}} t+i\delta) - \exp(i\delta)\right] \\
y &= y_0 \pm {\color{red}{(v_\perp/\omega_c)}} \left[ \exp(i{\color{green}{\omega_c}} t+i\delta) - \exp(i\delta) \right] \\
z &= z_0 + v_{z,0}t
\end{align}
The quantity $\boxed{\color{red}{r_L \equiv \frac{v_\perp}{\omega_c}}}$ is the **Larmor radius** or **gyro-radius**.

## Practice:

Let's represent the movement of two imaginary particles, with $(q,m)$ values respectively $(-1,1)$ and $(0.3,30)$ [*Note that $m_p/m_e \approx 1\,836$*]. For that we convert the Lorentz force equation in a system of first order differential equations,
\begin{align}
\dot{\vec{r}}(t) &= \vec{v}(t) \\
\dot{\vec{v}}(t) &= \frac{q}{m}\vec{v}(t)\times\vec{B}
\end{align}
and we solve this system with a numerical algorithm.

We start by importing some numerical libraries...

In [ ]:
%matplotlib
# Computation functions:
import numpy as np
from scipy.integrate import odeint
# Visualization routines:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.style.use('ggplot')

Then we define a function for the system,

In [ ]:
"""Movement of electron and ion under a constant magnetic field"""

def EqMovement(Q,t,QbyM):
    """Equations of movement.
    
    Parameters
    ----------
    
    Q : list 
        The (x,y,z) components of position and velocity
    t : float
        Time value for time dependent functions. Not used in this case but required by odeint
    QbyM : float
        The ratio between charge and mass (q/m).
    """
    
    global B0
    v = Q[3:]
    drdt = v                                    # Velocity
    dvdt = QbyM*np.cross(v,B0)                  # Acceleration
    return np.concatenate((drdt,dvdt))
  
def trajectories(Q0, QbyM, tf):
    """Computes the electron and proton trajectories.
    
    Parameters
    ----------
    
    Q0 : list
        The (x,y,z) components of the initial position and velocity
    QbyM : list
        The ratio between charge and mass (q/m) for the two charges.
    tf : int
        The final time.
    """
    
    t = np.linspace(0,tf,5*tf)
    # Integration of the equations of movement
    Qe = odeint(EqMovement, Q0, t, args=(QbyM[0],))
    Qp = odeint(EqMovement, Q0, t, args=(QbyM[1],))
    return Qe, Qp

def initFigure():
    """Sets the figure for the trajectories"""
    
    line1.set_data([], []); line1b.set_data([], [])
    line2.set_data([], []); line2b.set_data([], [])
    # Legibility
    ax.set_title("Trajectories",fontsize=18)
    ax.set_xlabel("X Axis",fontsize=16); ax.set_ylabel("Y Axis",fontsize=16)
    ax.text(18,8,"$\\odot\\,\\vec{B}$", fontsize=20)
    return line1, line1b, line2, line2b

def animate(i,re,rp):
    """Plots the trajectories in (x,y) for i"""
    
    line1b.set_data(re[:i,0],re[:i,1]); line1.set_data(re[i,0],re[i,1])
    line2b.set_data(rp[:i,0],rp[:i,1]); line2.set_data(rp[i,0],rp[i,1])
    return line1, line1b, line2, line2b

# --- Main ---  

global B0

# Constants
q = 1; me = 0.3; Mp = 30*me; Bz = 2
# Initial conditions
v0z = 0
r0 = np.zeros(3); v0 = np.array([0,2,v0z])  # Initial position and velocity
B0 = np.array([0,0,Bz])                     # Magnetic fields

# Frequency and radius
we = q*B0[2]/me; wp = q*B0[2]/Mp            # Cyclotron frequencies    
modv = np.sqrt(v0[0]**2+v0[1]**2)           # Perpendicular|B velocity
rLe = me/q*modv/Bz; rLp = Mp/me*rLe         # Larmor radius

print('Cyclotron frequencies      Larmor radius\n   we = {0:4.2f}, wp = {1:4.2f}  \
rLe = {2}, rLp = {3}'.format(we,wp,rLe,rLp))

tf = 150
# Computes the trajectories
QbyM = np.array([-q/me,q/Mp])               # Charge by mass ratio
Q0 = np.concatenate((r0,v0))                # Initial values
Qe,Qp = trajectories(Q0, QbyM, tf)          # Integrate the equations of movement

# Plots the trajectories
fig = plt.figure(figsize=(9,8))
ax = fig.add_subplot(111, autoscale_on=False, xlim=(-5,25), ylim=(-15,15))

line2, = ax.plot([], [], 'ro', markersize=12); line2b, = ax.plot([], [], 'r--')
line1, = ax.plot([], [], 'bo'); line1b, = ax.plot([], [], 'b--')

ani = animation.FuncAnimation(fig, animate, np.linspace(0,tf,5*tf), fargs=(Qe,Qp),
                              blit=False, interval=1,
                              repeat = False, init_func=initFigure)
#ani.save('uniformB.mp4', fps=20)
a = plt.show()

## Magnetic Moment

Let's recall the definition of magnetic moment of a current flowing in a closed circuit:
$$ \mu = I A,$$
where $I$ is the current and $A$ the area of the loop. In our case we have,
$$ I = |q|\frac{\omega_c}{2\pi},\quad A = \pi r_L^2 $$
and, as $r_L=v_\perp/\omega_c$ substituting values, 
$$ \boxed{\color{blue}{\mu = \frac{|q|v_\perp^2}{2\omega_c} = \frac{m v_\perp^2}{2 B} = \frac{W_\perp}{B}}}. $$


# $\vec{E} \neq \vec{0}$, constant

In this case we need to solve the full Lorentz equation:
$$ \vec{a} = \frac{q}{m}\left(\vec{E}+\vec{v}\times\vec{B}\right). $$

## How is the movement?

$\color{red}{Discussion...}$

Let's take $\vec{E} = E_{\perp,0}\vec{u}_\perp+E_{\parallel,0}\vec{u}_z$ and $\vec{B}$ along z. We obtain
\begin{align}
\dot{v}_x &= \frac{q}{m}(E_x + v_y B) \\
\dot{v}_y &= \frac{q}{m}(E_y - v_x B) \\
\dot{v}_z &= \frac{q}{m}E_\parallel
\end{align}
In the direction parallel to $\vec{B}$, we just have the free-fall in the electric field:
$$v_z = \frac{q}{m}E_\parallel t+v_{\parallel,0}.$$
In order to find a solution in the plane $\perp$ to $\vec{B}$, as in the previous case, we diferentiate the two equations and substitute the first derivatives to obtain the two equations
\begin{align}
\ddot{v}_x &= -\omega_c^2(v_x - \frac{E_y}{B}) \\
\ddot{v}_y &= -\omega_c^2(v_y + \frac{E_x}{B}) \\
\end{align}
Making the substitution $u_x = v_x-E_y/B,\; u_y = v_y+E_x/B$, we obtain for $u_x,\;u_y$ the same equations as in the previous case! I.e., in xy the movement can be decomposed on a Larmor movement and a drift component.

*Note:* If we want to procede in a more formal way, we take again the derivative of any of the first two equations we obtain the *inhomogeneous equation for a harmonic oscilator*,
$$ \ddot{v}_{x,y} + \omega_L^2 v_{x,y} = \omega_L^2 \frac{E_{y,x}}{B}, $$
whose solution is the solution for the *homogeneous* case plus a particular solution
$\Rightarrow \vec{v}_\perp$ has a Larmor movement with a drift: $v_\perp = v_\bigcirc + v_d$.

**How to compute $v_d$?**

If we average the Lorentz force over many gyroperiods, the average acceleration is *zero* and the only velocity component left is $v_d$:
$$ 0 = \frac{q}{m}(\vec{E}+\vec{v}_d\times\vec{B}). $$
Taking the cross product with $\vec{B}$ and using the vector formula in the appendix, we finally obtain

$$ \vec{v}_{d} = \frac{\vec{E}\times\vec{B}}{B^2}$$

## Summary

* The drift is $\perp$ both to $\vec{E}$ and $\vec{B}$;
* On the **same direction** for electrons and ions $\Rightarrow$ no net current!;
* The drift is independent of m, q and $v_\perp$.
    * Exercise: What is the $\vec{E}_\perp$ field in an inertial frame moving with $\vec{v}_d$? (*Hint*: use the Lorentz transformation for an electromagnetic field.)
    
---

## Practice:

We can easily extend the previous example to include the electric field. It is also interesting to see what happens with the kinetic energy and the Larmor radius.

We include one more library to allow us to interact with the script and we leave the rendering of the figure to an external script.

In [ ]:
%matplotlib inline
from ipywidgets import interact
from scripts import plotEB

modv = lambda v: np.sqrt((v[:,0])**2+(v[:,1])**2)    # v-perpendicular

def EqMovement(Q,t,QbyM):
    """Equations of movement"""
    global E0,B0
    v = Q[3:]
    drdt = v                                    # Velocity
    dvdt = QbyM*(E0+np.cross(v,B0))             # Acceleration
    return np.concatenate((drdt,dvdt))

def crossEB(Exy=0, Ez=0, angle=320):
    """Movement of an electron and ion under constant 
    magnetic and electric fields"""
    global E0,B0
    
    me = 1; Mp = 10*me; q = 1; Bz = 1 
    QbyM = np.array([-q/me,q/Mp])               # Charge by mass ratio

    r0 = np.zeros(3); v0 = np.array([0,1,0])
    Q0 = np.concatenate((r0,v0))                # Initial values
    E0 = np.array([Exy,Exy,Ez]); B0 = np.array([0,0,Bz])

    tf = 350; NPts = 5*tf; t = np.linspace(0,tf,NPts)
    # Computes the trajectories
    Qe,Qp = trajectories(Q0, QbyM, tf)          # Integrate the equations of movement
    
    # Drift velocity
    vd = np.cross(E0,B0)/np.dot(B0,B0)            # vd = (EXB)/(B|B)
    rd = np.array([t,vd[0]*t,vd[1]*t]).T
    
    # Larmor radius
    rLe = me/q*modv(Qe[:,3:5])/Bz; rLp = Mp/q*modv(Qp[:,3:5])/Bz
    
    # Kinetic energy
    We = me/2*(Qe[:,3]**2+Qe[:,4]**2+Qe[:,5]**2)
    Wp = Mp/2*(Qp[:,3]**2+Qp[:,4]**2+Qp[:,5]**2)
    
    # Plot the trajectories and Larmor radius
    plotEB.plot3d(Qe, Qp, rd, t, rLe, rLp, We, Wp, angle)

dummy = interact(crossEB, Exy=(0,1), Ez=(0,0.2), angle=(180,360))

# General force, $\vec{F}$

The result above can be generalized for any constant and uniform force such that $\vec{F}\cdot\vec{B} = 0$:

$$ \vec{v}_{d} = \frac{\vec{F}\times\vec{B}}{q B^2}$$

In particular, for the gravitational field, we have a drift $v_g = \frac{m}{q}\frac{\vec{g}\times\vec{B}}{B^2}$. In this case **the direction of drift depends on the signal of** $q \Rightarrow$ for positive and negative charges, we have a current!

The velocity of any particle can be decomposed in three components:
$$ \vec{v} = \vec{v}_\parallel+\vec{v}_{d}+\vec{v}_{L} $$

# Appendix - Useful vector formulae

$$ \vec{A}\times(\vec{B}\times\vec{C}) = (\vec{A}\cdot\vec{C})\vec{B}-(\vec{A}\cdot\vec{B})\vec{C} $$